# nucleus

> core functionalities of this lib

In [1]:
#| default_exp nucleus

In [2]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

## 实验初始化

In [3]:
# | export
from flowillower.help import lib_directory_path, runs_path
from scholarly_infrastructure import logger, print
app_template_path = lib_directory_path / 'flowillower_app_template'

In [4]:
import scholarly_infrastructure
from scholarly_infrastructure import logger
logger.info("test")
print("test")

Tue 2025-05-27 21:42:54.886868

INFO     test                                                                                       ]8;id=479399;file:///tmp/ipykernel_1722333/1853071160.py\1853071160.py]8;;\:]8;id=858202;file:///tmp/ipykernel_1722333/1853071160.py#3\3]8;;\

Tue 2025-05-27 21:42:54.892943

INFO     test                                                                                         ]8;id=260161;file:///home/ye_canming/repos/novelties/ScholarlyInfrastructure/scholarly_infrastructure/logging/nucleus.py\nucleus.py]8;;\:]8;id=942945;file:///home/ye_canming/repos/novelties/ScholarlyInfrastructure/scholarly_infrastructure/logging/nucleus.py#65\65]8;;\

In [5]:
app_template_path.exists()

True

In [6]:
# | export

from sympy import Not
from scholarly_infrastructure import experiment_setting, RandomVariable
from typing import Optional, List
from git import Repo, InvalidGitRepositoryError, GitCommandError
from pathlib import Path
import shutil
@experiment_setting
class VisualizationApp:
    app_path: str = ~RandomVariable(
        "./flowillower_app", description="Local directory to store the visualization app"
    )
    app_remote_url: Optional[str] = ~RandomVariable(
        None, description="Optional remote Git repository URL for pushing changes"
    )
    app_remote_branch: str = ~RandomVariable(
        "main", description="Branch name for the remote repository"
    )
    remote_url_is_new: bool = ~RandomVariable(
        True, description="Whether the remote URL is a new repository to be created"
    )
    @property
    def app_data_path(self):
        """
        Returns the path to the app data directory.
        """
        return Path(self.app_path).resolve() / "data"
    def setup(self):
        """
        Sets up the visualization app by cloning the remote repository if provided.
        """
        app_path = Path(self.app_path).resolve()
        if app_path.exists():
            return # 默认创建好了，不管用户有没有自由地修改 app.py 和 git 信息
        print(f"[flowillower] Creating app directory at {app_path} ")
        # 从 template 得到这个app
        # 直接复制一份 app_template_path 过去
        if not self.remote_url_is_new:
            raise NotImplementedError(
                "Remote URL is not new. This feature is not implemented yet."
            )
        
        shutil.copytree(app_template_path, self.app_path, dirs_exist_ok=True)
        
        print(f"[flowillower] App directory created at {app_path} ")
        # TODO
        # if self.app_remote_url: # 用户希望这个 app 在 这个 remote url 上看到
        #     # 修改 remote url 即可
        #     try:
        #         repo = Repo(app_path)
        #         origin.set_url(self.app_remote_url)
        #         print(f"[flowillower] Remote URL set to {self.app_remote_url}.")
        #     except GitCommandError as e:
        #         raise RuntimeError(f"Failed to set remote URL: {e}")
        # else:
        #     print("[flowillower] No remote URL provided. Skipping remote setup.")

In [7]:
app = VisualizationApp(
    (runs_path/"test_app").as_posix(),
    "https://huggingface.co/spaces/THU-CVML/flowillower_app"
)
app.setup()
repo = Repo((runs_path/"test_app").as_posix())
repo.remotes

[]

In [8]:
#| export
import math

WORD_MAP = [
    ["垂柳", "烟柳", "杨柳", "柳浪", "柳荫"],    # 柳 0
    ["雾隐", "幽谷", "深潭", "暮色", "云蔽"],    # 暗 1
    ["桃李", "芳菲", "芙蕖", "槿荣", "菊韵"],    # 花 2
    ["晨曦", "晴岚", "霁色", "皎月", "星河"]     # 明 3
]

def number_to_phrase(n: int, sep: str = "-") -> str:
    """将数字转换为无限扩展的成语组合"""
    if n < 0:
        raise ValueError("输入必须是非负整数")

    # 生成动态进制编码
    digits = []
    while n > 0:
        digits.append(n % 5)
        n = n // 5
  
    # 生成词汇链
    components = []
    for i, d in enumerate(digits):
        layer = i % 4  # 0=柳 1=暗 2=花 3=明
        word_set = WORD_MAP[layer]
        components.append(word_set[d % 5])
  
    # 从最高位开始拼接
    return sep.join(reversed(components)) if components else WORD_MAP[0][0]
# TODO 似乎不太对。


In [9]:
[number_to_phrase(i) for i in range(10)]


[
    '垂柳',
    '烟柳',
    '杨柳',
    '柳浪',
    '柳荫',
    '幽谷-垂柳',
    '幽谷-烟柳',
    '幽谷-杨柳',
    '幽谷-柳浪',
    '幽谷-柳荫'
]

In [10]:
#| export
from datetime import datetime

@experiment_setting
class Trial(VisualizationApp):
    """
    Represents a trial within an institution, inheriting from Institution.
    """
    # 以后再做仓库解耦，先把写入逻辑搞定
    study_name:str  =~RandomVariable("study")
    trial_name: Optional[str] =~RandomVariable(None)
    trial_description: str = ~RandomVariable(
        "A description of the trial", description="Description of the trial"
    )
    input_variables: dict =~RandomVariable(
        default_factory=dict, description="Input variables for the trial"
    )
    trial_start_date: datetime = ~RandomVariable(datetime.now(), description="Start date of the trial")
    trial_end_date: Optional[datetime] = ~RandomVariable(
        None, description="End date of the trial, if applicable"
    )
    participants: List[str] = ~RandomVariable(default_factory=list, description="List of participants in the trial")
    setup_immediately:bool =~RandomVariable(True, description="")

            
    # report_format:str = "flowillower/toml", # 我们这个框架定义的格式
    report_to: List[str] = ~RandomVariable(
        default_factory = lambda:[
        # "flowillower/csv", 
        # "tensorboard", # 本地
        # "wandb", # 
        # "mlflow" # 开源
        ],
        description="List of platforms to report experiment results to",
    )
    is_on_main_process: bool = ~RandomVariable(
        True, description="Whether the trial is running on the main process, if not, it will not report results. "
    )

    def setup(self):
        if not self.is_on_main_process:
            # TODO 应该读取文件获得信息，确保信息正确。
            return
        self.trial_start_date = datetime.now()

        # if "flowillower/toml" in self.report_to:
        super().setup() # 确保app仓库存在了
        trial_path = self.trial_path
        trial_path.mkdir(parents=True, exist_ok=True)

        if "wandb" in self.report_to:
            # try:
            import wandb
            # except:
                # raise ImportError("wandb is not installed. ")
            self.third_parties["wandb"] = wandb.init(
                project=self.study_name, 
                name=self.trial_name, 
                notes=self.trial_description
            )
        if "swanlab" in self.report_to:
            import swanlab
            self.third_parties["swanlab"] = swanlab.init(
                project=self.study_name, 
                experiment_name = self.trial_name,
                description = self.trial_description
            )
            
            
    


    @property
    def study_path(self) -> Path:
        return self.app_data_path/self.study_name

    def random_new_trial_name(self) -> str:
        study_path = self.study_path
        # 有多少个 子文件夹
        if study_path.exists():
            existing_trials = [d.name for d in study_path.iterdir() if d.is_dir()]
            return number_to_phrase(len(existing_trials))
        else:
            return number_to_phrase(0)
    
    @property
    def trial_path(self) -> Path:
        assert self.trial_name is not None
        return self.study_path/self.trial_name

    def __post_init__(self):
        self.third_parties = dict()
        if self.trial_name is None:
            self.trial_name = self.random_new_trial_name()
        if self.setup_immediately:
            self.setup()
            

In [11]:
# 一层是数据，一层是vis app，这样huggingface不会触发重新加载。app局部的pull
# TODO 

In [ ]:
test_trial = Trial(app_path=(runs_path/"test_trial").as_posix(),
      study_name="test_study",
      report_to=["wandb", "swanlab"])

默认单体 TODO

In [86]:
# def create_trial():

In [19]:
import dataclasses
import argparse
from collections.abc import Mapping

def everything_to_dict(obj):
    """
    将类似字典的对象（如argparse.Namespace、dataclass等）递归转换为标准字典。
  
    支持类型：
    - 字典及所有Mapping类型（如OrderedDict）
    - dataclass实例
    - argparse.Namespace实例
    - 普通类实例（含__dict__属性）
    - 使用__slots__的类实例
    - namedtuple
    - 列表、元组、集合（递归处理元素）
    """
    # 处理字典及其他Mapping类型（如OrderedDict）
    if isinstance(obj, Mapping):
        return {k: everything_to_dict(v) for k, v in obj.items()}
  
    # 处理dataclass实例
    if dataclasses.is_dataclass(obj):
        return everything_to_dict(dataclasses.asdict(obj))
  
    # 处理argparse.Namespace
    if isinstance(obj, argparse.Namespace):
        return everything_to_dict(vars(obj))
  
    # 处理普通类实例（含__dict__）
    if hasattr(obj, '__dict__'):
        return everything_to_dict(vars(obj))
  
    # 处理使用__slots__的类实例
    if hasattr(obj, '__slots__'):
        slots = obj.__slots__ if isinstance(obj.__slots__, (list, tuple)) else [obj.__slots__]
        return {slot: everything_to_dict(getattr(obj, slot)) for slot in slots if hasattr(obj, slot)}
  
    # 处理列表、元组、集合（递归处理元素）
    if isinstance(obj, (list, tuple, set)):
        return [everything_to_dict(e) for e in obj]
  
    # 处理namedtuple
    if isinstance(obj, tuple) and hasattr(obj, '_fields') and hasattr(obj, '_asdict'):
        return everything_to_dict(obj._asdict())
  
    # 基本类型直接返回
    return obj

## 超参数记录

In [22]:
#| export
from fastcore.all import patch
import tomli_w
@patch
def write_input_variables(self:Trial):
    if not self.is_on_main_process:
        return
    report_path = self.trial_path / "input_variables.toml"
    report_path.parent.mkdir(parents=True, exist_ok=True)
    with open(report_path, "wb") as f:
        tomli_w.dump(self.input_variables, f)

@patch
def report_input_variables(self:Trial, input_variables:dict, wandb_config_important=True)->dict:
    """
    Report the trial arguments to the specified platforms.
    """
    if not self.is_on_main_process:
        return
    
    if "wandb" in self.report_to:
        import wandb
        wandb_dict = wandb.config.as_dict()
        if wandb_config_important:
            input_variables = {**input_variables, **wandb_dict}# 压轴出手
        else:
            input_variables = {**wandb_dict, **input_variables}
    
    self.input_variables |= input_variables

    if "wandb" in self.report_to:
        import wandb
        wandb.config.update(self.input_variables)
        # wandb.config.update(allow_val_change=True)
        # args = wandb.config # type: ignore

    if "swanlab" in self.report_to:
        import swanlab
        swanlab.config.update(self.input_variables)

    self.write_input_variables()
    return self.input_variables

In [23]:
test_trial.report_input_variables(dict(a=1, b=2, c=3))
test_trial.report_input_variables(dict(d=4))

{'a': 1, 'b': 2, 'c': 3, 'd': 4}

## 指标上传

### 异步step一致性算法

In [24]:
#| export
from typing import Optional, List, Set

all_tracks:Set['Track'] = set()
max_global_step = -1
@experiment_setting
class Track:
    """
    A class to represent a phase for logging.
    """
    name: str =~ RandomVariable("default")
    description:str =~RandomVariable("")
    local_step:int =~RandomVariable(-1)
    global_step:int =~RandomVariable(-1)

    def step_once(self):
        """
        每次`report`的时候，先调用这个函数确保对齐了，然后再读取这个对象的 global_step
        """
        self.local_step += 1 # 自己这个phase内的step很好决定。
        # 关键是要根据现在场上已有的其他phase来决定自己的global_step是什么。
        global all_phases, max_global_step
        # max_global_step = max(phase.global_step for phase in all_phases)
        if self.global_step==max_global_step:
            self.global_step += 1 # 我是最大者，我观察到了新的值，意味着时间发生了变化，我来定义 global step。 
            max_global_step = self.global_step # 记录下来，供其他人使用。
        else:
            self.global_step = max_global_step # 同步到其他人的最大值，我和他们是同时的，我落后于他们的记录，我记录的对象的性质和他们记录的对象的性质是一样的，理应没有发生变化。

train_track = Track(name="train")
val_track = Track(name="val")
test_track = Track(name="test")

logging

In [25]:
#| export
from fastcore.basics import patch
from filelock import FileLock
from pathlib import Path

import tomli

@patch
def report_step_third_party(
    self: Trial,
    information: dict,
    global_step: int,
    track_name: str = "default"
):
    postfix = f"-{track_name}"
    information = {f"{k}{postfix}": v for k, v in information.items()}
    if "wandb" in self.report_to:
        import wandb
        wandb.log(information, step=global_step)
    if "swanlab" in self.report_to:
        import swanlab
        swanlab.log(information, step=global_step)



@patch
def report_step(
    self: Trial,
    information: dict,
    global_step: int,
    track_name: str = "default"
):
    """
    Report one step's data to the specified track.

    - `data`: dict of metric name → value
    - `global_step`: unified training step
    - `track_name`: name of the logical phase (e.g. "train", "val", "test")

    Each track has its own TOML file using [[metrics]] array of tables.
    """
    if not self.is_on_main_process:
        return
    scalar_dir = self.trial_path / "logs/scalar"
    scalar_dir.mkdir(parents=True, exist_ok=True)

    file_name = f"metrics_{track_name}.toml"
    report_path = scalar_dir / file_name

    entry = {"global_step": global_step, **information}

    if not report_path.exists():
        # First write: create file with initial entry
        with open(report_path, "wb") as f:
            tomli_w.dump({"metrics": [entry]}, f)
    else:
        # Append new entry by reading old, then writing back
        with open(report_path, "rb") as f:
            old_data = tomli.load(f)

        metrics = old_data.get("metrics", [])
        metrics.append(entry)

        with open(report_path, "wb") as f:
            tomli_w.dump({"metrics": metrics}, f)
    
    self.report_step_third_party(information, global_step, track_name)
    



@patch
def report_step_lock(
    self: Trial,
    information: dict,
    global_step: int,
    track_name: str = "default"
):
    """
    Efficiently append a step's data to a TOML log file under a named track.

    Uses low-level file append + file lock for speed and thread-safety.
    """
    scalar_dir = self.trial_path / "logs/scalar"
    scalar_dir.mkdir(parents=True, exist_ok=True)

    file_name = f"metrics_{track_name}.toml"
    report_path = scalar_dir / file_name
    lock_path = report_path.with_suffix(".lock")

    # Format the data as TOML string manually
    lines = ["\n[[metrics]]", f"global_step = {global_step}"]
    for k, v in information.items():
        if isinstance(v, str):
            lines.append(f'{k} = "{v}"')
        elif isinstance(v, bool):
            lines.append(f"{k} = {'true' if v else 'false'}")
        else:
            lines.append(f"{k} = {v}")
    lines.append("")  # final newline
    content = "\n".join(lines)

    # Use file lock for safe concurrent writing
    lock = FileLock(lock_path)
    with lock:
        with open(report_path, "a", encoding="utf-8") as f:
            f.write(content)



@patch
def report(self:Trial, information:dict, track:Track, step_forward:bool = True):
    """
    Report the trial arguments to the specified platforms.
    """
    if step_forward:
        track.step_once()
    return self.report_step(information, track.global_step, track_name=track.name)

@patch
def report_train(self:Trial, information:dict, step_forward:bool = True):
    """
    Report the trial arguments to the training track.
    """
    return self.report(information, train_track, step_forward)
@patch
def report_val(self:Trial, information:dict, step_forward:bool = True):
    """
    Report the trial arguments to the validation track.
    """
    return self.report(information, val_track, step_forward)
@patch
def report_test(self:Trial, information:dict, step_forward:bool = True):
    """
    Report the trial arguments to the test track.
    """
    return self.report(information, test_track, step_forward)


In [26]:
loss = 30
acc = 0.5
for epoch in range(10):
    for batch in range(20):
        loss = loss*0.8
        acc = acc*1.1
        test_trial.report_train(dict(loss=loss, acc=acc))
    test_trial.report_val(dict(loss=loss*1.2, acc=acc/1.2))
test_trial.report_test(dict(loss=loss*1.3, acc=acc/1.3))

## 生态外推

自动跟随 wandb.log

In [ ]:
# TODO

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()